In [244]:
# imports
import pandas as pd

# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import json
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# cnn imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.layers.normalization import BatchNormalization
#from tensorflow.keras.layers.advanced_activations import LeakyReLU

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import Layer
from keras.layers.embeddings import Embedding

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [186]:
# Load data
bodies_df = pd.read_csv("train_bodies.csv")
stances_df =  pd.read_csv("train_stances.csv")

In [187]:
# Examine data shape
bodies_df.shape

(1683, 2)

In [188]:
stances_df.shape

(49972, 3)

In [189]:
combined_df = pd.DataFrame(columns=['Headline', 'articleBody', 'Stance'])
combined_df.head()

,Headline,articleBody,Stance


In [190]:
for i in range (1250):
    combined_df.at[i, 'Headline'] =  stances_df.at[i,'Headline']
    #combined_df.at[i,'articleBody'] = bodies_df.at[i,'articleBody' ]
    #combined_df.at[i,'articleBody'] = bodies_df.loc[bodies_df['Body ID'] == stances_df.at[i,'Body ID'], 'articleBody']
    combined_df.at[i,'articleBody'] = bodies_df.loc[bodies_df['Body ID'] == stances_df.at[i,'Body ID'], ['articleBody']].iloc[0]['articleBody']
    combined_df.at[i,'Stance'] = stances_df.at[i,'Stance']

In [191]:
combined_df.head()

,Headline,articleBody,Stance
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,agree
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,unrelated
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",disagree


In [192]:
combined_df.to_csv("combined_stances.csv")

In [193]:
# store column names for convenience
column_names = combined_df.columns
display(column_names)

Index(['Headline', 'articleBody', 'Stance'], dtype='object')

In [194]:
# Extract column for just labels
labels = combined_df['Stance']

In [195]:
# Drop columns that are strings
all_data_processed = combined_df.drop(['Stance'], axis=1)

In [196]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = all_data_processed[1000:], labels[1000:]
#dev_data, dev_labels = all_data_processed[17505:23340], labels[17505:23340]
train_data, train_labels = all_data_processed[:1000], labels[:1000]

In [197]:
# Used GLoVE embeddings
embeddings_dict = {}
with open("glove.6B.50d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        token = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[token] = vector

In [198]:
embeddings_dict['time']

array([ 0.02648  ,  0.33737  ,  0.065667 , -0.11609  ,  0.41651  ,
       -0.21142  , -0.69582  ,  0.2822   , -0.36077  , -0.13822  ,
        0.012094 ,  0.086227 , -0.84638  ,  0.057195 ,  1.1582   ,
        0.14703  , -0.0049197, -0.24899  , -0.96014  , -0.3038   ,
        0.23972  ,  0.21058  ,  0.40608  ,  0.17789  ,  0.55253  ,
       -1.6357   , -0.17784  , -0.45222  ,  0.45805  ,  0.14239  ,
        3.7087   ,  0.40289  , -0.4083   , -0.29304  ,  0.030857 ,
       -0.15361  ,  0.10607  ,  0.63397  ,  0.12397  , -0.25349  ,
       -0.10344  ,  0.0069768, -0.17328  ,  0.35536  , -0.46369  ,
        0.15285  ,  0.41475  , -0.3398   , -0.23043  ,  0.19069  ],
      dtype=float32)

In [199]:
import gensim

In [200]:
# NLTK for NLP utils
import nltk
# Helper libraries.
#from w266_common import utils, vocabulary, tf_embed_viz
import utils, vocabulary, tf_embed_viz

In [250]:
# define documents
docs = ['Well done!',
'Good work',
'Great effort',
'nice work',
'Excellent!',
'Weak',
'Poor effort!',
'not good',
'poor work',
'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 8
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[2, 15], [8, 30], [45, 36], [24, 30], [29], [40], [10, 36], [46, 8], [10, 30], [38, 37, 15, 1]]
[[ 2 15  0  0  0  0  0  0]
 [ 8 30  0  0  0  0  0  0]
 [45 36  0  0  0  0  0  0]
 [24 30  0  0  0  0  0  0]
 [29  0  0  0  0  0  0  0]
 [40  0  0  0  0  0  0  0]
 [10 36  0  0  0  0  0  0]
 [46  8  0  0  0  0  0  0]
 [10 30  0  0  0  0  0  0]
 [38 37 15  1  0  0  0  0]]


In [378]:
def sents_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    
    bad_chars = [';', ':', '!', "*"] 
    
    text_ids = []
    for sentence in sentences:
        example = []
        sentence = sentence.replace('\'','')
        sentence = sentence.replace(',','')
        sentence = sentence.replace('$','')
        sentence = sentence.replace('/',' ')
        sentence = sentence.replace('','')
        for word in sentence.split(' '):
            #word.lower()
            temp = word.lower()
            #temp.replace('\'','')
            if temp in embeddings_dict:
                example.append(embeddings_dict[temp])
            #else:
                #print("Missing ", temp)            
        text_ids.append(example)

    return  text_ids   

In [380]:
blah = combined_df['Headline']

In [419]:
blah

0       Police find mass graves with at least '15 bodi...
1       Hundreds of Palestinians flee floods in Gaza a...
2       Christian Bale passes on role of Steve Jobs, a...
3       HBO and Apple in Talks for $15/Month Apple TV ...
4       Spider burrowed through tourist's stomach and ...
                              ...                        
1245    Was Heart-Tugging Viral Video Of Generous Home...
1246    Tourist wakes to find strange red trail on his...
1247    ISIS Fighters Getting Caught Coming Across the...
1248    Mass grave found after 40 students disappear i...
1249    Missing Mexican students 'not found in mass gr...
Name: Headline, Length: 1250, dtype: object

In [381]:
train_input = np.array(sents_to_ids(blah))

In [426]:
train_input[0][17].shape

(50,)

In [431]:
train_input[12]

[array([ 0.94397  , -0.67342  ,  1.0082   , -0.44745  ,  0.8761   ,
        -0.0081819, -1.6734   ,  1.2498   , -0.13068  , -0.40058  ,
         0.55603  , -0.74259  , -0.059912 ,  0.49491  ,  0.89407  ,
        -0.71861  , -0.56199  ,  0.022498 , -0.062564 ,  1.0018   ,
        -0.052287 ,  1.6375   ,  0.84925  ,  1.0139   , -0.27091  ,
        -1.3237   , -0.15111  , -0.82478  , -0.10847  ,  0.11021  ,
         1.0944   , -0.67153  , -0.63257  ,  0.2961   ,  0.58242  ,
         0.020359 , -0.75819  , -1.1431   ,  0.14229  ,  0.10008  ,
        -0.32434  ,  0.11604  ,  1.1102   , -1.1168   ,  1.0355   ,
        -0.99955  , -1.1571   , -0.10277  ,  0.11576  , -0.9225   ],
       dtype=float32),
 array([-0.36647 , -0.7656  ,  0.13563 ,  0.313   ,  0.37232 , -0.87062 ,
        -0.50308 ,  0.32128 ,  0.77395 ,  0.35854 ,  0.7704  ,  0.27343 ,
         0.27201 ,  0.27854 ,  0.16601 , -0.3523  ,  0.081415,  0.50749 ,
        -0.27102 ,  0.23128 ,  0.18316 ,  0.8289  , -0.15401 ,  0.11125 ,


In [411]:
num_steps = 1
hidden_size = 50

model2 = Sequential()
#model2.add(Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32'))
#model.add(Embedding(vocab, hidden_size, input_length=num_steps))
model2.add(Embedding(vocab_size, hidden_size, input_length=num_steps))
model2.add(LSTM(hidden_size, return_sequences=True))
model2.add(LSTM(hidden_size, return_sequences=True))
model2.add(Dropout(0.5))
#model.add(TimeDistributed(Dense(vocab_size)))
model2.add(Activation('softmax'))

In [412]:
# summarize the model
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 1, 50)             2500      
_________________________________________________________________
lstm_31 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
lstm_32 (LSTM)               (None, 1, 50)             20200     
_________________________________________________________________
dropout_16 (Dropout)         (None, 1, 50)             0         
_________________________________________________________________
activation_14 (Activation)   (None, 1, 50)             0         
Total params: 42,900
Trainable params: 42,900
Non-trainable params: 0
_________________________________________________________________
None


In [415]:
model2.predict(train_input[0][1])

array([[[0.02000602, 0.01999915, 0.01999659, ..., 0.01997498,
         0.0199929 , 0.01999348]],

       [[0.02002586, 0.01999705, 0.01999787, ..., 0.02000447,
         0.01999288, 0.01999779]],

       [[0.02002586, 0.01999705, 0.01999787, ..., 0.02000447,
         0.01999288, 0.01999779]],

       ...,

       [[0.02002586, 0.01999705, 0.01999787, ..., 0.02000447,
         0.01999288, 0.01999779]],

       [[0.02002586, 0.01999705, 0.01999787, ..., 0.02000447,
         0.01999288, 0.01999779]],

       [[0.02002586, 0.01999705, 0.01999787, ..., 0.02000447,
         0.01999288, 0.01999779]]], dtype=float32)

In [416]:
model2.compile(optimizer='adam', loss='binary_crossentropy')

In [418]:
model2.fit(train_input[0][1], labels )

ValueError: Error when checking target: expected activation_14 to have 3 dimensions, but got array with shape (10, 1)